In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import caffe

%matplotlib gtk

In [19]:
def detect_faces(image):
    height, width, depth = image.shape
    classifier = cv2.CascadeClassifier('Detector_XML/haarcascade_frontalface_alt.xml')
    DOWNSCALE = 4
    minisize = (int(width/DOWNSCALE), int(height/DOWNSCALE))
    smallimg = cv2.resize(image, minisize)
    faces = classifier.detectMultiScale(smallimg)
    if(len(faces) > 0):
        print("Foram econtradas %s faces" % len(faces))
        for face in faces:
            x, y, w, h = [ coordinate*DOWNSCALE for coordinate in face ]
            color = (255, 255, 0)
            point1 = (x, y)
            point2 = (x + w, y + h)
            cv2.rectangle(frame, point1, point2, color)
    else:
        print("Não foram encontradas faces!")
    return frame

In [20]:
def setup():
    classifier_path = 'Detector_XML/haarcascade_frontalface_alt.xml'
    

In [24]:
cap = cv2.VideoCapture(0)

while(True):
    ret, frame = cap.read()
    img = frame.copy()
    frame = detect_faces(frame)
    cv2.imshow('frame', frame)
    if((cv2.waitKey(1) & 0xFF) == ord('q')):
        break

cap.release()
cv2.destroyAllWindows()

Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
Foram econtradas 1 faces
